## Connection

In [1]:
from web3 import Web3
from datetime import datetime
import pandas as pd
import json

In [2]:
infura_url= 'https://mainnet.infura.io/v3/[YOUR API KEY]'
web3= Web3(Web3.HTTPProvider (infura_url))
web3.isConnected()

True

## Dissecting Transaction

In [3]:
address= web3.toChecksumAddress("0x1ad91ee08f21be3de0ba2ba6918e714da6b45836")
transaction= '0x032ed60363beb809a2b9c9790bb7dadd83b743040945a087aeecbe9e6b2dc2af'

* Nonce

In [4]:
next_nonce= web3.eth.getTransactionCount(address)
next_nonce

3460174

In [5]:
transaction_nonce= web3.eth.get_transaction(transaction).nonce
transaction_nonce

3460171

* Gas

In [6]:
gasPrice= web3.eth.get_transaction(transaction).gasPrice
gasPrice

19209576968

In [7]:
gaslimit= web3.eth.get_transaction(transaction).gas
gaslimit

100000

In [8]:
gasUsed= web3.eth.getTransactionReceipt (transaction).gasUsed
gasUsed

21000

* To

In [9]:
recipient= web3.eth.get_transaction(transaction).to
recipient

'0x3c16183c1C0E28F1a0cB9F8ee4b21D0Db208cA46'

* Value

In [10]:
value= web3.eth.get_transaction(transaction).value
value*10**-18

48.0

* Data

In [11]:
data= web3.eth.get_transaction(transaction).input
data

'0x'

In [12]:
transaction_ba='0xb3827bb3cca1a693ec69edb744755f64d8ff8c90f89f69cbfbfafd17b0083159'

In [13]:
data_ba= web3.eth.get_transaction(transaction_ba).input
data_ba

'0xa723533e0000000000000000000000000000000000000000000000000000000000000001'

In [14]:
bored_ape= web3.eth.get_transaction(transaction_ba)

In [15]:
sc_address="0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"
with open ("./ba_abi.json") as f:
    abi= json.load (f)

In [16]:
baContract = web3.eth.contract(address=sc_address, abi=abi)

In [17]:
function, parameters = baContract.decode_function_input(bored_ape["input"])
print (function, 'parameters: ',parameters )

<Function mintApe(uint256)> parameters:  {'numberOfTokens': 1}


* Log events

In [18]:
abi_events = [abi for abi in baContract.abi if abi["type"] == "event"]
for event in abi_events:
    name = event["name"]
    inputs = [param["type"] for param in event["inputs"]]
    inputs = ",".join(inputs)
    event_abi_human = f"{name}({inputs})"
    event_abi_hex = web3.toHex(web3.keccak(text=event_abi_human))

In [19]:
txreceipt= web3.eth.getTransactionReceipt (transaction_ba)
for l in range (0, len (txreceipt['logs'])):
    event_hex = web3.toHex(txreceipt['logs'][l]['topics'][0])
    if event_abi_hex == event_hex:
        decoded_logs = baContract.events[event["name"]]().processReceipt(txreceipt)
decoded_logs

(AttributeDict({'args': AttributeDict({'from': '0x0000000000000000000000000000000000000000',
   'to': '0x9909017A0F637380af916257D05c3e7dD2F6c68a',
   'tokenId': 6633}),
  'event': 'Transfer',
  'logIndex': 124,
  'transactionIndex': 321,
  'transactionHash': HexBytes('0xb3827bb3cca1a693ec69edb744755f64d8ff8c90f89f69cbfbfafd17b0083159'),
  'address': '0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D',
  'blockHash': HexBytes('0x26537966b001e26c3c960bfb619d1af5fdd4d04c54596fb50a3a94d73bf66a69'),
  'blockNumber': 12347114}),)